In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import json
import dateutil.parser
from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)

past_last_date = False

C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = []
page_num = 0

ticker = 'ndaq'

sector = get_sector(ticker)

index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)


while True:
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://ir.nasdaq.com/news-and-events/press-releases?page={page_num}', headers=headers)
    

    soup = BeautifulSoup(html.content) 
    content = soup.find('tbody')
    if content == None:
        break

    rows = content.find_all('tr')
    for row in rows:
        date = row.find('td', attrs={'data-before':'Date'}).text.lstrip().rstrip() 
        title = row.find('a', attrs={'class':'nir-widget--accordion-toggle nir-widget--news--accordion-toggle'}).text

        url = row.find('a', attrs={'hreflang':'en'}).get('href')
        
        url = f'https://ir.nasdaq.com{url}'
        
        date = dateutil.parser.parse(date).date()

        content = row.find('div', attrs={'class':'nir-widget--field nir-widget--news--teaser'}).text.lstrip().rstrip()
        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break        
    
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    if past_last_date == True: break    
    page_num = page_num + 1
    
save_data(data, ticker, sector)

past_last_date = False